In [1]:
from gnews import GNews
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
google_news = GNews(language='es', 
                    country='Mexico', 
                    period='7d', 
                    start_date=None, 
                    end_date=None, 
                    max_results=5, 
                    #exclude_websites=['yahoo.com', 'cnn.com'],
                    #proxy=proxy
                    )
news = google_news.get_news('espanol Claudia Sheinbaum')

In [12]:

def find_article_body_complete(soup):
    # A more comprehensive list of strategies to capture various HTML structures
    strategies = [
        {'selector': 'div.article-body', 'method': 'select'},
        {'selector': 'article', 'method': 'find'},
        {'selector': 'div.article-content', 'method': 'select'},
        {'selector': 'main.content', 'method': 'select'},
        {'selector': 'div#article-text', 'method': 'select'},
        {'selector': 'p', 'method': 'find_all'},  # Fallback to gather all paragraphs
    ]
    
    article_parts = []
    
    for strategy in strategies:
        if strategy['method'] == 'select':  # For CSS selectors
            elements = soup.select(strategy['selector'])
        else:  # For find_all methods
            elements = soup.find_all(strategy['selector'])
            
        for element in elements:
            text = element.get_text(separator="\n", strip=True)
            if text:
                article_parts.append(text)
                
        # If we found some content, no need to continue with further strategies
        if article_parts:
            break
    
    # Combine all parts of the article we found
    return "\n\n".join(article_parts) if article_parts else None

def parse_article_content(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)
    print(response)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        article_text = find_article_body_complete(soup)

        if not article_text or "Please enable JS and disable any ad blocker" in article_text or len(article_text) < 100:
            try:
                chrome_options = Options()
                chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Specify the correct Chrome binary location
                driver_executable_path = "/home/uumami/.wdm/drivers/chromedriver/linux64/122.0.6261.111/chromedriver-linux64/chromedriver"
                driver = webdriver.Chrome(service=Service(executable_path=driver_executable_path), options=chrome_options)

                driver.get(url)
                driver.implicitly_wait(10)  # Wait for content to load
                html = driver.page_source
                driver.quit()  # Close the browser

                soup = BeautifulSoup(html, 'html.parser')
                article_text = find_article_body_complete(soup)
                return article_text if article_text else "Failed to parse the article content with Selenium."
            except Exception as e:
                print(f"Failed to parse the article with Selenium due to an error: {e}")
                return None
    else:
        print("Failed to retrieve the article using requests.")
        return None


In [15]:
google_news = GNews(language='es', 
                    country='Mexico', 
                    period='7d', 
                    start_date=None, 
                    end_date=None, 
                    max_results=5, 
                    #exclude_websites=['yahoo.com', 'cnn.com'],
                    #proxy=proxy
                    )
news = google_news.get_news('espanol Claudia Sheinbaum')

In [16]:
# Example usage - replace the URL with the one you're interested in
url = 'https://news.google.com/rss/articles/CBMiVmh0dHBzOi8vd3d3Lm55dGltZXMuY29tL2VzLzIwMjQvMDMvMDEvZXNwYW5vbC9lbGVjY2lvbmVzLW1leGljby1jbGF1ZGlhLXNoZWluYmF1bS5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en'
print(url)
article_content = parse_article_content(url)
if article_content:
    print("Article Content:", article_content)
else:
    print("Failed to parse the article content.")

https://news.google.com/rss/articles/CBMiVmh0dHBzOi8vd3d3Lm55dGltZXMuY29tL2VzLzIwMjQvMDMvMDEvZXNwYW5vbC9lbGVjY2lvbmVzLW1leGljby1jbGF1ZGlhLXNoZWluYmF1bS5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en
<Response [200]>
Article Content: Failed to parse the article content with Selenium.


In [6]:
# Example usage - replace the URL with the one you're interested in
url = news[1]['url']
print(url)
article_content = parse_article_content(url)
if article_content:
    print("Article Content:", article_content)
else:
    print("Failed to parse the article content.")

https://news.google.com/rss/articles/CBMijwFodHRwczovL3d3dy5sb2NhbDEwLmNvbS9lc3Bhbm9sLzIwMjQvMDMvMDEvYXJyYW5jYS1jYW1wYW5hLXBvci1sYS1wcmVzaWRlbmNpYS1kZS1tZXhpY28tZW50cmUtb2ZlcnRhcy1kZS1jb250aW51aWRhZC15LWNyaXRpY2FzLWEtbGEtdmlvbGVuY2lhL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en


03/07/2024 03:20:41 PM - ====== WebDriver manager ======
/bin/sh: line 1: google-chrome: command not found
/bin/sh: line 1: google-chrome: command not found
03/07/2024 03:20:41 PM - Get LATEST chromedriver version for google-chrome
03/07/2024 03:20:41 PM - Get LATEST chromedriver version for google-chrome
03/07/2024 03:20:41 PM - Driver [/home/uumami/.wdm/drivers/chromedriver/linux64/122.0.6261.111/chromedriver-linux64/chromedriver] found in cache


Article Content: Failed to parse the article content with Selenium.
